Esta práctica consiste en la detección de ciertas zonas de interés en coches. Está compuesta por tres apartados y en
cada uno se deberá desarrollar un fichero. La práctica deberá ejecutarse sobre Python 3.7.X y OpenCV 4.2. y  los
ficheros serán los siguientes:

- deteccion_orb.py
- detección_haar.py
- deteccion_video.py.

Para ejecutar la práctica deberá escribirse en la consola de comandos `python` seguido del nombre del fichero sin ningún
otro parámetro adicional. Se supondrá que los directorios test, train y haar están en el mismo directorio que los
ficheros python. Al ejecutar estos ficheros python se mostrará por pantalla el resultado sobre cada una de las imágenes
o vídeos de test.

# Apartado 1: Detección de coches mediante puntos de interés

Este apartado aparece desarrollado en el fichero *deteccion_orb.py*.

Inicialmente se importan las librerías necesarias para la ejecución del programa y se comprueba que las versiones de
Python y OpenCV son las correctas. Se debe usar la versión 3.7 de Python y la versión 4.2 de OpenCV.

In [17]:
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
import sys
import math
import time

assert (sys.version.startswith('3.7')), "No se esta usando la version 3.7 de Python. Version en uso: " + sys.version
assert (cv2.__version__.startswith('4.2')), "No se esta usando la version 4.2 de OpenCV. Version en uso: " + cv2.__version__

Si se están usando las versiones adecuadas, no se mostrará nada por pantalla. En otro caso, se mostrará un mensaje de
error indicando la versión que se está usando y la que se debe usar.

*sys* permite comprobar la versión de Python, _os_ será útil a la hora de cargar las imágenes de un directorio,
*cv2* se usará para cargar y procesar las imágenes, _pyplot_ permite mostrar las imágenes, *numpy* es útil para
manipular las matrices correspondientes a las imágenes, *math* se usará para realizar operaciones aritméticas y
trigonométricas y, por último, se medirá el rendimiento de la ejecución mediante _time_.

A continuación, se definen algunos métodos para la carga de las imágenes.

El método `load()` recibe una cadena de texto como parámetro que indica el nombre del directorio en el que se encuentran
las imágenes a cargar. El directorio por defecto es _train_, ya que este método se usará siempre que se quiera entrenar
el sistema y contiene las imágenes de entrenamiento. Este método obtiene la ruta en la que se encuentra el fichero
actual y concatena la cadena de texto pasada como parámetro para obtener la ruta completa del directorio de
entrenamiento. A continuación, lista los ficheros
que se encuentran en el directorio mediante el método `os.listdir()`. Este método devuelve una lista de cadenas de texto
con los nombres de los archivos que se encuentran en el directorio en un orden aleatorio. Para mantener el orden de las 
imágenes se llama al método `ordenar()`. Por último, se carga cada una de estas imágenes mediante el método
`cv2.imread()`, que recibe la ruta de cada imagen del directorio y el flag 0, que indica que la imagen se carga en
niveles de gris, y se devuelve una lista de matrices con todas las imágenes del directorio.

El método `load_color()` carga las imágenes igual que el método anterior, pero manteniendo el color de  las imágenes. La
diferencia es que en el momento de la carga no se emplea el flag 0.

El método `ordenar()` recibe una lista de cadenas de texto. El método `sort()` para listas de Python ordena la lista
del siguiente modo: 

>['frontal_1.jpg', 'frontal_10.jpg', 'frontal_11.jpg', ... , 'frontal_19.jpg', 'frontal_2.jpg', 'frontal_20.jpg', ...,]

Pero sería más adecuado mantener las imágenes en un orden natural (0, 1, 2, 3, ...). Para ello, primero se ordenan las
cadenas por longitud, de modo que las imágenes de los frontales del 1 al 10 quedarían al principio. Se obtienen
estas cadenas de texto y se ordenan, se ordena el resto por separado y, por último, se concatenan y se devuelven.

In [18]:
def ordenar(lst):
    """Recibe una lista y la devuelve ordenada"""
    lst.sort(key=len)
    ret = lst[0:10]
    ret.sort()
    aux = lst[10:]
    aux.sort()
    return ret + aux


def load(directory='train'):
    """Recibe el nombre de un directorio y devuelve una lista con las imagenes contenidas en el"""
    cur_dir = os.path.abspath(os.curdir)
    files = ordenar(os.listdir(cur_dir + '/' + directory))
    return [cv2.imread(directory + '/' + file, 0) for file in files]


def load_color(directory):
    """Recibe el nombre de un directorio y devuelve una lista con las imagenes contenidas en el a color"""
    cur_dir = os.path.abspath(os.curdir)
    files = ordenar(os.listdir(cur_dir + '/' + directory))
    return [cv2.imread(directory + '/' + file) for file in files]

También hay un método `soft_load()` que carga solamente 6 imágenes del directorio _train_.

In [1]:
def soft_load():
    """Devuelve una lista con 6 imagenes preseleccionadas aleatoriamente"""
    return [cv2.imread('train/frontal_9.jpg', 0), cv2.imread('train/frontal_39.jpg', 0),
            cv2.imread('train/frontal_43.jpg', 0), cv2.imread('train/frontal_7.jpg', 0),
            cv2.imread('train/frontal_19.jpg', 0), cv2.imread('train/frontal_26.jpg', 0)]

Inicialmente se implementó otro método `load2()` para cargar las imágenes. Este método concatena una cadena de texto
que contiene una parte común de la ruta de todas las imágenes de test con un número del 1 al 49. De este modo, es
posible cargar las 48 imágenes de test, pero si se añaden más imágenes al directorio, estas no se cargarán.
En cambio, el método
`load()` sí lo haría. Por este motivo y porque ambos métodos tienen un tiempo de ejecución similar se decidió usar el
método `load()`.

In [20]:
def load2():
    """Devuelve una lista con las 48 imagenes de entrenamiento"""
    return [cv2.imread('train/frontal_' + str(i) + '.jpg', 0) for i in range(1, 49)]


A continuación, se implementan unos métodos `calculate_module()` y `calculate_angle_to_centre()` que serán necesarios más
adelante para calcular el módulo del vector que une un punto con el centro de las imágenes de entrenamiento y el ángulo
en grados que forma dicho vector respecto a la porción positiva del eje X en sentido horario, respectivamente. Ambos
métodos reciben una tupla con las coordenadas del punto como parámetro. Pueden recibir un segundo punto para construir
el vector. Si no lo reciben, se calcula el vector con un punto preestablecido, considerado el centro del frontal.

In [22]:
def calculate_module(p, centre=(225, 110)):
    """Recibe dos puntos y devuelve el modulo del vector que los une"""
    return np.sqrt((centre[0] - p[0]) ** 2 + (centre[1] - p[1]) ** 2)


def calculate_angle_to_centre(p, centre=(225, 110)):
    """Recibe dos puntos y devuelve el angulo del vector que los une"""
    return (math.atan2((p[1] - centre[1]), (centre[0] - p[0])) * 180 / math.pi) % 360

También se crea una clase _Match_ para almacenar los puntos de interés aprendidos durante el entrenamiento. En esta
clase se almacena el módulo del vector que une el punto de interés con el centro de la imagen, el ángulo que forma este
vector como se ha indicado antes, la escala del punto de interés y el ángulo del punto de interés respecto a la imagen.

In [23]:
class Match:
    def __init__(self, module, kp_angle, scale, des_angle):
        self.module = module
        self.kp_angle = kp_angle
        self.scale = scale
        self.des_angle = des_angle

    def get_module(self):
        """Devuelve el modulo del vector que une el punto de interes con el centro de la imagen"""
        return self.module

    def get_kp_angle(self):
        """Devuelve el angulo del vector que une el punto de interes con el centro de la imagen"""
        return self.kp_angle

    def get_scale(self):
        """Devuelve la escala del punto de interes"""
        return self.scale

    def get_des_angle(self):
        """Devuelve el angulo del punto de interes respecto de la imagen"""
        return self.des_angle

En un primer momento, el entrenamiento y la detección se realizaban dentro de un mismo método. Más tarde se observó que
sería necesario utilizar estas operaciones en otros apartados, por lo que se decidió desacoplar el entrenamiento de la
detección, implementándolos en métodos distintos.

El método `train()` corresponde al entrenamiento del sistema. Este método recibe una lista de imágenes y un detector de
puntos de interés y descriptores y devuelve una tabla de objetos *Match* con los puntos de interés aprendidos y una
estructura de datos de tipo flann entrenada con los descriptores de las imágenes de entrenamiento. En el método se crea
una estructura de datos tipo *FlannBasedMatcher* que sirve como árbol de búsqueda. En el constructor se indica que el 
algoritmo de búsqueda sea _LSH_ y que se busque en todas las hojas del árbol. Se inicializa la tabla de puntos de 
interés aprendidos *match_table* y se recorre la lista de imágenes recibida. Para cada imagen se obtienen los puntos
de interés (_kps_) y descriptores (*des*).

A continuación, para cada punto de interés, se almacenan en una lista auxiliar
*image_match* los objetos _Match_ con la información necesaria para la detección. El flann es capaz de encontrar los
descriptores más parecidos, pero además, hay que "enseñar" dónde está el centro de la imagen y para ello está la tabla
*match_table*. Por cada punto de interés encontrado, se debe guardar un vector que apunte hacia el centro del frontal.
Por ejemplo, si se encuentra el faro derecho del coche, el vector deberá apuntar a la izquierda, ya que ahí se
encontraría el centro del frontal. De este modo, cuando se encuentre un faro derecho en una imagen de test, la tabla
indicará que el centro está a la izquierda porque ya lo ha aprendido. Ya que las imágenes están centradas y cuadradas
en el frontal de los coches de entrenaniento, se asumirá que los centros de los frontales corresponden a los centros de
las imágenes. Todas las imágenes tienen el mismo tamaño y, por tanto, un centro común. Es por este motivo, que las
funciones `calculate_module()` y `calculate_angle_to_centre()` calculan lso vectores respecto a dicho centro común, el
punto (225,110).

La lista auxiliar se introduce en la
tabla y se añade una lista con los descriptores de la imagen al *flann* mediante el método `add()`. Es importante
destacar que se deben añadir los descriptores de la imagen dentro de una lista, debido a que cada vez que se llama al
método, la lista pasada como parámetro se concatena con una lista con los descriptores ya recibidos. Para que la
búsqueda funcione correctamente y se devuelvan las distancias entre descriptores e índices correctos, se deben
separar los descriptores de las distintas imágenes.

In [24]:
def train(images, detector):
    """Devuelve una tabla con los puntos de interes aprendidos y un arbol flann entrenado"""
    FLANN_INDEX_LSH = 6
    index_params = dict(algorithm=FLANN_INDEX_LSH, table_number=6, key_size=3, multi_probe_level=1)
    search_params = dict(checks=-1)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    match_table = []
    for image in images:
        kps, des = detector.detectAndCompute(image, None)
        image_match = [Match(calculate_module(k.pt), calculate_angle_to_centre(k.pt), k.size, k.angle) for k in kps]
        match_table.append(image_match)
        flann.add([des])

    return match_table, flann

El método `detect()` corresponde a la fase de detección en las imágenes de test. Recibe como
parámetros una lista con las imágenes test, un detector de puntos de interés y descriptores, la tabla con los puntos de 
interés aprendidos, el flann entrenado, el
número de emparejamientos a devolver y la sigma del kernel gaussiano; y devuelve una lista de tuplas con las coordenadas
de los centros de los frontales de cada imagen de test. Adicionalmente, recibe un parámetro *debug*. Si este parámetro
vale 1 se crearán listas para almacenar los descriptores, puntos de interés y matrices de votación de cada imagen de
test para facilitar la depuración. Primero se define una lista *detected_points* que almacenará los puntos a
devolver. A continuación, se
recorre la lista de imágenes recibida y para cada imagen se obtienen los puntos de interés y los descriptores. Se
realiza una llamada a la función `knnMatch()`, que busca los _k_ descriptores más cercanos a los de la imagen de test.
La función devuelve los índices de la imágen de entrenamiento en la que se ha encontrado el emparejaminto
(_imgIdx_), del descriptor de dicha imagen con el cual se ha emparejado (_trainIdx_) y del descriptor de testing
emparejado (_queryIdx_). Una vez se tienen los emparejamientos, se recorren obteniendo los índices mencionados
anteriomente. Es importante tener en cuenta que los descriptores de las imágenes están directamente relacionados con
los puntos de interés a través de los índices. Esto significa que si los puntos de interés se han almacenado en orden
correctamente según se cargaban, los índices obtenidos servirán para localizar los puntos de interés aprendidos que
corresponden a los descriptores emparejados y que estaban almacenados en la tabla *match_table*.
Con _imgIdx_ se accede a la fila de la tabla *match_table* en la que se encuentran los puntos de interés de la imagen
de entrenamiento emparejada y, mediante _trainIdx_, se accede al punto de interés concreto con el que se emparejó.
Mediante _queryIdx_ se obtiene aquel punto de interés de la imagen de test con el que se emparejó. Al localizar el
punto de interés de entrenamiento se obtiene el objeto _Match_, el cual contiene el módulo y el ángulo del vector que,
partiendo de las coordenadas del punto de interés, señala el punto a buscar; en este caso, el frontal del coche. Sin
embargo, se debe tener en cuenta la relación entre las escalas y los ángulos de los puntos de interés emparejados.
La diferencia
entre las escalas modificará el módulo del vector para adaptarlo a la imagen de test y una simple operación aritmética
devolverá el ángulo final del vector. Multiplicando el coseno del ángulo por el nuevo módulo y sumándo el resultado a la
coordenada x del punto de interés de la imagen test se obtiene la coordenada x del punto en el que se encontraría el
frontal. Ya que la
matriz de votación tiene una resolución 10 veces menor que la imagen, se divide el resultado entre 10 y se eliminan los
decimales (ya que la matriz solo acepta índices enteros). Se calcula el punto de la otra coordenada operando esta vez
con el seno y se añade 1 voto a lamatriz de votación controlando que no se salga de los límites. El coseno y el seno
calculados tienen tales signos para calcular los ángulos respecto a la porción positiva del eje x.

Al terminar el bucle se dispondrá de una matriz con una celda (o varias) con valor máximo que indicará en qué celda se
encuentra el frontal. Es posible que la matriz tenga dos zonas de valores máximos o focos, ambos con el mismo valor
máximo, pero que uno de ellos tenga votos altos agrupados y el otro un único voto máximo. En estos casos es posible que
se escoja el punto que no corresponde al centro, y por ello hay que tener en cuenta no solo los valores máximos
puntuales sino la agrupación de valores altos, ya que estos últimos corresponderán al centro. Para ello, se
convoluciona la matriz con un kernel gausiano que permitirá localizar los focos más grandes. En la matriz suavizada se
aprecia cómo las celdas agrupadas aumentan su valor y hacen que el valor de la central destaque sobre el de la celda
solitaria.

Para obtener los índices de dicha celda se puede usar la función `unravel_index()` de Numpy. La tupla obtenida
se multiplica por 10 para volver a la resolución inicial de la imagen de test y se añadirá a la lista declarada
al inicio. Para finalizar, se devuelve la lista de tuplas con las coordenadas de los centros de los frontales de
las imágenes de test.

In [25]:
def detect(images, detector, match_table, flann, knn_matches, sigma, debug):
    """Devuelve una lista de tuplas con las coordenadas de los puntos detectados"""
    if debug == 1:
        test_kps_table = []
        test_des_table = []
        matrices_votacion = []
    detected_points = []

    for test_image in images:
        kps, des = detector.detectAndCompute(test_image, None)
        if debug == 1:
            test_des_table.append(des)
            test_kps_table.append(kps)

        results = flann.knnMatch(des, k=knn_matches)

        matriz_votacion = np.zeros((int(test_image.shape[0] / 10), int(test_image.shape[1] / 10)), dtype=np.float32)

        for r in results:
            for m in r:
                match = match_table[m.imgIdx][m.trainIdx]
                m_test = kps[m.queryIdx]
                trns = (m_test.size / match.get_scale()) * match.get_module()
                angle = match.get_kp_angle() + match.get_des_angle() - m_test.angle
                x = int((m_test.pt[0] + (trns * math.cos(angle))) / 10)
                y = int((m_test.pt[1] - (trns * math.sin(angle))) / 10)
                if 0 < x < matriz_votacion.shape[1] and 0 < y < matriz_votacion.shape[0]:
                    matriz_votacion[y, x] += 1

        ksize = 6 * sigma + 1
        kernel_y = cv2.getGaussianKernel(ksize, sigma)
        kernel_x = kernel_y.T
        matriz_filtrada = cv2.sepFilter2D(matriz_votacion, -1, kernel_y, kernel_x)

        if debug == 1:
            matrices_votacion.append(matriz_filtrada)

        z = np.unravel_index(np.argmax(matriz_filtrada, axis=None), matriz_filtrada.shape)
        q = (int(z[1] * 10), int(z[0] * 10))
        detected_points.append(q)

    return detected_points

Se ha decidido desacoplar la detección de puntos de la presentación de los resultados una vez más para permitir el uso
de estos métodos en otros apartados. El método `draw_points()` recibe una lista de tuplas con coordenadas de puntos y
una lista de imágenes sobre las que mostrar estos puntos. Se recorren ambas listas dibujando un círculo de color rojo,
radio 15 y grosor 10 mediante la función `cv2.circle()` y mostrándo la imagen en pantalla.

In [26]:
def draw_points(images, points):
    """Dibuja un circulo en los puntos dados sobre las imagenes recibidas como parametro"""
    for index in range(len(images)):
        cv2.circle(images[index], points[index], 15, (255, 0, 0), thickness=10, lineType=8, shift=0)
        plt.imshow(cv2.cvtColor(images[index], cv2.COLOR_RGB2BGR))
        plt.title("Imagen " + str(index + 1))
        plt.show()

Todos estos métodos se invocan en la función `main()`. Esta recibe unos parámetros que indican, respectivamente, el
número de puntos de interés a detectar en cada imagen, el factor de escala entre los distintos niveles, y los niveles,
de la pirámide del detector, el número de emparejamientos a calcular, la sigma del kernel gausiano y, por último, un
entero que facilita la depuración del programa.

Se comienza almacenando las imágenes de entrenamiento en *train_images* y creando un detector ORB mediante
`cv2.ORB_create()`. El detector creará una pirámide de imágenes a distinta escala de la cual obtendrá
los puntos de interés y los descriptores. La diferencia de escala entre cada imagen de la pirámide sera *scale_factor*
y el número de imágenes será *pyramid_levels*, y el número de descriptores devuelto será *num_keypoints*.
A continuación, se llama a la función de entrenamiento, que recibe los dos objetos
creados y devuelve un árbol de búsqueda flann entrenado y una tabla con los puntos de interés aprendidos. El siguiente
paso es cargar las imágenes de test, se cargarán en niveles de gris en una lista, y en color en otra. A continuación,
se invoca la función de detección, que devolverá las
coordenadas en las que se encuentran los frontales de las imágenes de test. Para acabar, se muestra la localización de
los frontales llamando a la función `draw_points()`.

In [27]:
def main(num_keypoints, scale_factor, pyramid_levels, knn_matches, gaussian_kernel_sigma, debug=0):
    train_images = load()
    orb = cv2.ORB_create(nfeatures=num_keypoints, scaleFactor=scale_factor, nlevels=pyramid_levels)
    match_table, flann = train(train_images, orb)
    test_images = load('test')
    test_images_color = load_color('test')
    # para hacer deteccion de una imagen en concretro, pasar esta imagen en una lista del siguiente modo
    # test_images = [test_images[i]], donde i es el indice de la imagen a testear
    detected_points = detect(test_images, orb, match_table, flann, knn_matches, gaussian_kernel_sigma, debug)
    draw_points(test_images_color, detected_points)

Por último, se introduce una estructura de control para llamar a la función `main()` y ejecutar el código cuando se
ejecute el mandato `python deteccion_orb.py`. De este modo, también es posible importar el fichero en otro programa e
invocar los métodos definidos. Se han declarado unas constantes que representan todas las variables de las que depende
el programa.

In [28]:
if __name__ == "__main__":
    NUM_KEYPOINTS = 500
    SCALE_FACTOR = 1.3
    PYRAMID_LEVELS = 4
    KNN_MATCHES = 6
    GAUSSIAN_KERNEL_SIGMA = 2
    DEBUG = 1

    # para ver las matrices de votacion, introducir el parametro DEBUG
    #main(NUM_KEYPOINTS, SCALE_FACTOR, PYRAMID_LEVELS, KNN_MATCHES, GAUSSIAN_KERNEL_SIGMA)

Cuando se ejecute el fichero, aparecerán las imágenes de test con los frontales marcados de uno en uno. La imagen no
desaparecerá hasta que se cierre la ventana o se pulse la tecla *q*. En ese momento aparecerá la siguiente imagen.

# Apartado 2:Detección de coches usando cv2.CascadeClassifier

Este apartado aparece desarrollado en el fichero *deteccion_haar.py*.

Al inicio del fichero se importan las librerías a utilizar. Cabe destacar que se importa el fichero desarollado en
el aparado anterior `import deteccion_orb as orbdet` para poder hacer uso del método `load()`.

Al igual que en el apartado anterior, se ha decidido hacer una aplicación modular para que los métodos definidos se
puedan usar en otros apartados.

In [29]:
import cv2
from matplotlib import pyplot as plt
import deteccion_orb as orbdet

Para desarrollar este apartado se ha consultado la documentación de OpenCV. En ella aparecía un ejemplo de código con
un fin muy similar al de este apartado, por lo que se decidió tomar ese código y adaptarlo.

El método `detect()` corresponde a la fase de detección del frontal y la matrícula de los coches. Recibe una lista de
imágenes en nivel de gris sobre las que detectar las zonas de interés, una lista con las mismas imágenes en color para
dibujar sobre ellas las zonas de interés, clasificadores para coches y matrículas y dos valores numéricos a utilizar en
la detección. Se declara una lista con las imágenes a devolver y
se recorren las imágenes en gris detectando zonas de interés, en este caso frontales de coche, mediante el método
`detectMultiScale()`. Este método recibe como parámetros la imágen sobre la que detectar, el factor
de escala entre las imágenes generadas (similar a como hacía ORB con la pirámide) y el número mínimo de vecinos que
debe tener una zona candidata para ser válida. La función devuelve las coordenadas del rectángulo que representa la zona
en la que se encuentra el frontal del coche (los dos primeros enteros indican las coordenadas de la esquina superior
izquierda y los otros dos, la anchura y altura, respectivamente). La función `cv2.rectangle()`  recibe la imagen en
coloy y las esquinas superior izquierda e inferior derecha del rectángulo y lo dibuja con el color y grosor dados.
La detección de la mátrícula se podría hacer de dos
maneras: buscándola en la región delimitada por el frontal (ya que la matrícula está en él) o en toda la imagen. Se ha
realizado la detección de ambas formas para ver el comportamiento del algoritmo. Almacenando la región en la que se
encuentra el frontal en las variables _roi_ se busca la matrícula del mismo modo que antes y se dibuja un rectángulo
más grueso para diferenciar la detección de matrículas en el frontal de la detección en toda la imagen. Por último, se
busca en toda la imágen y la imagen con todas las zonas detectadas se guarda en la lista a devolver.

In [ ]:
def detect(gray_images, color_images, coches_cascade, matriculas_cascade, scale_factor, min_neighbors):
    """Detecta el frontal y la matricula de los coches y devuelve una lista de imágenes con las zonas detectadas"""
    detected_images = []

    for i in range(len(gray_images)):
        gray = gray_images[i]
        color = color_images[i]

        coche = coches_cascade.detectMultiScale(gray, scale_factor, min_neighbors)
        for (x, y, w, h) in coche:
            color = cv2.rectangle(color, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = color[y:y + h, x:x + w]
            matricula = matriculas_cascade.detectMultiScale(roi_gray, scale_factor, min_neighbors)
            for (ex, ey, ew, eh) in matricula:
                cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 4)

        matricula = matriculas_cascade.detectMultiScale(gray, scale_factor, min_neighbors)
        for (x, y, w, h) in matricula:
            color = cv2.rectangle(color, (x, y), (x + w, y + h), (0, 0, 255), 2)

        detected_images.append(color)

    return detected_images

El siguiente método simplemente recibe una lista de imágenes y las muestra por pantalla indicando de qué imagen se
trata. La función `cv2.color()` recibe una imagen y la transformación de color a realizar sobre ella. Las imágenes en
OpenCV se cargan en BGR y lo más común es que se muestren en RGB, por lo que se cambia el orden de los canales y se
muestra.

In [ ]:
def show(images):
    for i in range(len(images)):
        plt.imshow(cv2.cvtColor(images[i], cv2.COLOR_RGB2BGR))
        plt.title("Imagen " + str(i + 1))
        plt.show()

En el directorio *haar_opencv_4.1-4.2* se incluyen los ficheros _coches.xml_ y _matriculas.xml_, que corresponden a
clasificadores entrenados para la detección de coches y matrículas, respectivamente. Para utilizar estos clasificadores
es necesario invocar la función `cv2.CascadeClassifier()`, que recibirá un fichero xml y devolverá un clasificador
entrenado. En la función principal se crean dos clasificadores, uno para frontales y otro para matrículas. A
continuación, se cargan las imágenes en niveles de gris y en color y se llama a la función de detección. Esta
devolverá una lista de imágenes con las zonas de interés marcadas con un rectángulo. Para acabar se llama a la función
que muestra las imágenes por pantalla.

In [30]:
def main(scale_factor, min_neighbors):
    coches_cascade = cv2.CascadeClassifier('haar_opencv_4.1-4.2/coches.xml')
    matriculas_cascade = cv2.CascadeClassifier('haar_opencv_4.1-4.2/matriculas.xml')

    test_images_gray = orbdet.load('test')
    test_images_color = orbdet.load_color('test')

    frontales = detect(test_images_gray, test_images_color, coches_cascade, matriculas_cascade, scale_factor,
                       min_neighbors)
    show(frontales)

Al igual que en el primer apartado, las imágenes se mostrarán en ventanas separadas, que deberán cerrarse para ver la
siguiente imagen.

# Apartado 3: Detección del coche en secuencias de video

# Referencias
